In [20]:
%pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
"""
Objetos do sqlalchemy core para a realização da conexão e operação do Banco de Dados
"""
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

DB_CONNECTION = URL.create(
    drivername="postgresql",
    username="postgres",
    password="1234",
    host="10.8.0.13",
    port=5432,
    database="stemis"
)
engine = create_engine(DB_CONNECTION, echo=False)
Base = declarative_base()
Session = sessionmaker(bind=engine)

C:\Users\Gustavo\AppData\Local\Temp\ipykernel_22532\3574374962.py:18: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [22]:
from sqlalchemy import Column, Integer, String, Boolean, Float, DateTime

class Tcu(Base):
    __tablename__ = "tcu"
    id = Column(Integer, primary_key=True)
    name = Column(String)
    active = Column(Boolean)
    latitude = Column(Float)
    longitude = Column(Float) 

    def get_attr_dict(self):
        return {
            "id": self.id,
            "name": self.name,
            "active": self.active,
            "latitude": self.latitude,
            "longitude": self.longitude
        }


class TcuData(Base):
    __tablename__ = "tcu_data"
    id = Column(Integer, primary_key=True)  
    tcu_id = Column(String) 
    datetime = Column(DateTime)
    tcu_datetime = Column(DateTime)
    angular_position = Column(Float)
    target_angle = Column(Float)

    def get_attr_dict(self):
        return {
            "id": self.id,
            "tcu_id": self.tcu_id,
            "datetime": self.datetime.strftime("%Y-%m-%d %H:%M:%S.%f"),
            "tcu_datetime": self.tcu_datetime.strftime("%Y-%m-%d %H:%M:%S.%f"),
            "angular_position": self.angular_position,
            "target_angle": self.target_angle
        }


In [23]:
class Usina(object):
    def __init__(self):
        self._session = Session()

    def get_tcus(self):
        tcus = self._session.query(Tcu).all()
        data = [tcu.get_attr_dict() for tcu in tcus]
        return data
    

    def get_tcu_uplinks(self, dev_eui, n):
        uplinks = self._session.query(TcuData).filter(TcuData.tcu_id == dev_eui).order_by(TcuData.datetime.desc()).limit(n).all()
        data = [uplink.get_attr_dict() for uplink in uplinks]
        return data


In [24]:
usina = Usina()
tcus = usina.get_tcus()
tcus

[{'id': 'AC1F09FFFE1196B3',
  'name': 'TCU 14 - 0124090101840',
  'active': True,
  'latitude': -14.66588,
  'longitude': -57.51938},
 {'id': 'AC1F09FFFE119862',
  'name': 'TCU 07 - 0124090102908',
  'active': True,
  'latitude': -14.665909,
  'longitude': -57.51865},
 {'id': 'AC1F09FFFE1198BB',
  'name': 'TCU 19 - 0124090102854',
  'active': True,
  'latitude': -14.665868,
  'longitude': -57.519844},
 {'id': 'AC1F09FFFE1199DF',
  'name': 'TCU 16 - 0124090102182',
  'active': True,
  'latitude': -14.665883,
  'longitude': -57.519614},
 {'id': 'AC1F09FFFE11983C',
  'name': 'TCU 26 - 0124090100402',
  'active': True,
  'latitude': -14.665855,
  'longitude': -57.520392},
 {'id': 'AC1F09FFFE1198A8',
  'name': 'TCU 11 - 0124090102076',
  'active': True,
  'latitude': -14.665891,
  'longitude': -57.518994},
 {'id': 'AC1F09FFFE11992D',
  'name': 'TCU 12 - 0124090101024',
  'active': True,
  'latitude': -14.665873,
  'longitude': -57.519098},
 {'id': 'AC1F09FFFE1198D5',
  'name': 'TCU 24 - 012